## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
!pip install -U tensorflow --quiet

     |████████████████████████████████| 86.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 25.2MB/s 
     |████████████████████████████████| 450kB 47.2MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.6.3 which is incompatible.


In [2]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [66]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np


In [0]:
data = pd.read_csv('/gdrive/My Drive/Lab/Lab6/prices.csv')

In [120]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Check all columns in the dataset

In [122]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=["date","symbol"],axis=1,inplace=True)

In [124]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 5 columns):
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5)
memory usage: 32.5 MB


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
X=data.iloc[0:1000,0:4]

In [127]:
X.shape

(1000, 4)

In [0]:
y=data.iloc[0:1000,4]

In [0]:
y=y/1000000

In [130]:
y.shape

(1000,)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=26)

In [133]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 4)
(200, 4)
(800,)
(200,)


#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train=np.array(X_train.astype('float32'))
X_test=np.array(X_test.astype('float32'))
y_train=np.array(y_train.astype('float32'))
y_test=np.array(y_test.astype('float32'))


In [135]:
X_train.dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
scale = Normalizer()

X_train_z = scale.fit_transform(X_train)

X_test_z= scale.fit_transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [139]:
print(w)
print(b)

tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]], shape=(4, 1), dtype=float32)
tf.Tensor([0.], shape=(1,), dtype=float32)


2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)

    return y

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [143]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train_z, y_train, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train_z, w, b)).numpy())

Current Loss on iteration 0 187.45544
Current Loss on iteration 1 185.66652
Current Loss on iteration 2 184.01804
Current Loss on iteration 3 182.49866
Current Loss on iteration 4 181.09837
Current Loss on iteration 5 179.80795
Current Loss on iteration 6 178.61871
Current Loss on iteration 7 177.52267
Current Loss on iteration 8 176.51257
Current Loss on iteration 9 175.58177
Current Loss on iteration 10 174.7237
Current Loss on iteration 11 173.93298
Current Loss on iteration 12 173.20438
Current Loss on iteration 13 172.53285
Current Loss on iteration 14 171.91385
Current Loss on iteration 15 171.34348
Current Loss on iteration 16 170.81786
Current Loss on iteration 17 170.33328
Current Loss on iteration 18 169.88676
Current Loss on iteration 19 169.47537
Current Loss on iteration 20 169.0962
Current Loss on iteration 21 168.74663
Current Loss on iteration 22 168.42468
Current Loss on iteration 23 168.12775
Current Loss on iteration 24 167.85405
Current Loss on iteration 25 167.6020

### Get the shapes and values of W and b

In [144]:
print(w.shape)
print(w.numpy())

(4, 1)
[[1.2214307]
 [1.2253768]
 [1.2087715]
 [1.2360762]]


In [147]:
print(b.shape)
print(b.numpy())

(1,)
[2.4459636]


### Model Prediction on 1st Examples in Test Dataset

In [153]:
prediction(X_test_z, w, b)[0]

<tf.Tensor: id=32951, shape=(1,), dtype=float32, numpy=array([4.891755], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Iris=pd.read_csv('/gdrive/My Drive/Lab/Lab6/Iris.csv')

In [184]:
Iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
Iris=pd.get_dummies(Iris,columns=['Species'])

In [186]:
Iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
Id                         150 non-null int64
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), uint8(3)
memory usage: 6.4 KB


In [187]:
Iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [169]:
Iris['Species'].value_counts()

2    50
1    50
0    50
Name: Species, dtype: int64

### Splitting the data into feature set and target set

In [0]:
X=Iris.iloc[:,1:5]

In [190]:
X.shape

(150, 4)

In [0]:
y=Iris.iloc[:,5:]

In [198]:
y.shape

(150, 3)

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=26)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(4,)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [201]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 4)                 16        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________



### Model Training 

In [203]:
model.fit(X_train,y_train,          
          validation_data=(X_train,y_train),
          epochs=100,
          batch_size=16)

Train on 120 samples, validate on 120 samples
Epoch 1/100
120/120 [==============================] - 0s 234us/sample - loss: 0.4007 - accuracy: 0.8583 - val_loss: 0.3458 - val_accuracy: 0.9083
Epoch 2/100
120/120 [==============================] - 0s 183us/sample - loss: 0.4083 - accuracy: 0.8667 - val_loss: 0.3437 - val_accuracy: 0.9083
Epoch 3/100
120/120 [==============================] - 0s 173us/sample - loss: 0.3491 - accuracy: 0.9000 - val_loss: 0.3418 - val_accuracy: 0.9083
Epoch 4/100
120/120 [==============================] - 0s 228us/sample - loss: 0.4380 - accuracy: 0.8333 - val_loss: 0.3399 - val_accuracy: 0.9083
Epoch 5/100
120/120 [==============================] - 0s 173us/sample - loss: 0.3974 - accuracy: 0.8583 - val_loss: 0.3378 - val_accuracy: 0.9083
Epoch 6/100
120/120 [==============================] - 0s 180us/sample - loss: 0.3625 - accuracy: 0.8750 - val_loss: 0.3362 - val_accuracy: 0.9083
Epoch 7/100
120/120 [==============================] - 0s 156us/sample -

### Model Prediction

In [204]:
model.predict(X_test)

array([[6.51327753e-03, 5.11733532e-01, 4.81753200e-01],
       [2.53978241e-02, 8.40935290e-01, 1.33666813e-01],
       [9.72123444e-01, 2.75224168e-02, 3.54116521e-04],
       [9.94445801e-01, 5.47051337e-03, 8.36475738e-05],
       [1.96939986e-03, 2.81944931e-01, 7.16085672e-01],
       [1.88930612e-02, 9.53725815e-01, 2.73811370e-02],
       [9.52763915e-01, 4.69079986e-02, 3.28176829e-04],
       [8.89236617e-05, 4.96566817e-02, 9.50254381e-01],
       [1.33793865e-05, 4.59294766e-02, 9.54057217e-01],
       [1.07451505e-03, 3.26421529e-01, 6.72503889e-01],
       [9.90305662e-01, 9.47075430e-03, 2.23629846e-04],
       [1.01118460e-02, 6.31380498e-01, 3.58507663e-01],
       [2.20998637e-02, 9.11054969e-01, 6.68451265e-02],
       [9.56739962e-01, 4.30816785e-02, 1.78424045e-04],
       [2.13533901e-02, 9.03494060e-01, 7.51525909e-02],
       [9.84832942e-01, 1.51041402e-02, 6.29124333e-05],
       [2.48723896e-03, 3.39610666e-01, 6.57902062e-01],
       [7.71062225e-02, 8.90096

### Save the Model

In [0]:
model.save('R6_Iris_model.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?